###Тема: Рекомендаційні системи

## Завдання 1. Два алгоритми для рекомендаційної системи на основі датасетуMovieLens

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357287 sha256=13631706d5cc9de55e035d2d4a532d5d4c5148e4f2bdf8d078cb330a47548295
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, SVD, KNNBasic
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

In [ ]:
# 1. Завантажимо датасет ml-100k
data = Dataset.load_builtin('ml-100k')

In [ ]:
# 2. Виведемо перші 5 рядків завантаженого датасету
trainset, testset = train_test_split(data, test_size=0.2)
print('Перші 5 рядків датасету')
data.raw_ratings[:5]

Перші 5 рядків датасету


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596')]

In [ ]:
print('Перші 5 рядків датасету із поясненям даних')

for raw in data.raw_ratings[:5]:
  user_id, movie_id, rating, timestamp = raw
  print(f'Користувач {user_id} оцінив фільм №{movie_id} на {rating} балів (часова позначка: {timestamp})')

Перші 5 рядків датасету із поясненям даних
Користувач 196 оцінив фільм №242 на 3.0 балів (часова позначка: 881250949)
Користувач 186 оцінив фільм №302 на 3.0 балів (часова позначка: 891717742)
Користувач 22 оцінив фільм №377 на 1.0 балів (часова позначка: 878887116)
Користувач 244 оцінив фільм №51 на 2.0 балів (часова позначка: 880606923)
Користувач 166 оцінив фільм №346 на 1.0 балів (часова позначка: 886397596)


In [ ]:
# 3. Реалізуємо два алгоритми
algo1 = SVD()  # Алгоритм SVD
algo2 = KNNBasic()  # Алгоритм KNN Basic

# 4. Крос-валідація для підбору параметрів
cross_validate(algo1, data, measures=['MAE'], cv=5, verbose=True)
cross_validate(algo2, data, measures=['MAE'], cv=5, verbose=True)

# 5. Оберемо алгоритм з найнижчим MAE на основі результатів крос-валідації
algo1.fit(trainset)
predictions = algo1.test(testset)
print(f"SVD MAE: {accuracy.mae(predictions)}")

algo2.fit(trainset)
predictions = algo2.test(testset)
print(f"KNN MAE: {accuracy.mae(predictions)}")

Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7383  0.7387  0.7420  0.7352  0.7369  0.7382  0.0022  
Fit time          0.99    1.46    1.36    0.99    1.00    1.16    0.21    
Test time         0.26    0.15    0.10    0.10    0.25    0.17    0.07    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7701  0.7685  0.7673  0.7740  0.7841  0.7728  0.0061  
Fit time          0.22    0.23    0.23    0.31    0.23    0.24    0.03    
Test time

**Висновок**. Для алгоритму SVD маємо меншу помилку MAE (0,74), ніж для алгоритму KNN (0,77), тому приходимо до висновку що алгоритм SVD працює краще

Вивдемо далі рекомендації на основі алгоритму SVD для певного користувача

In [ ]:
len(data.raw_ratings)

100000

In [ ]:
for raw in data.raw_ratings[2357:2367]:
  user_id, movie_id, rating, timestamp = raw
  print(user_id)

83
280
43
18
303
91
318
275
97
174


In [ ]:
# 6. Знайдемо 10 рекомендацій для конкретного користувача
user_id = '83'
algo1.fit(trainset)
top_n = 10

recommendations = []
for i in range(1, 1683):  # 1682 - кількість фільмів
    prediction = algo1.predict(user_id, str(i))
    recommendations.append((i, prediction.est))

In [ ]:
# Виводимо топ-10 фільмів з найвищими оцінками
recommendations.sort(key=lambda x: x[1], reverse=True)
for movie_id, rating in recommendations[:top_n]:
    print(f"Фільм ID: {movie_id}, Оцінка: {rating}")

Фільм ID: 22, Оцінка: 4.757784908337378
Фільм ID: 318, Оцінка: 4.73961195611928
Фільм ID: 427, Оцінка: 4.6420085233829695
Фільм ID: 64, Оцінка: 4.60072355785519
Фільм ID: 12, Оцінка: 4.584725357177649
Фільм ID: 479, Оцінка: 4.555955781229809
Фільм ID: 79, Оцінка: 4.520907950080376
Фільм ID: 96, Оцінка: 4.518898376317491
Фільм ID: 408, Оцінка: 4.502835206152823
Фільм ID: 485, Оцінка: 4.493190392214228


Спробуємо тепер пошук параметрів по сітці для SVD тепер на основі RMSE

In [ ]:
from surprise.model_selection import GridSearchCV

In [ ]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=5)

gs.fit(data)

# найкращий RMSE score
print(gs.best_score["rmse"])

# комбіанція параметрів, яка дає найкращий RMSE score
print(gs.best_params["rmse"])

0.9607812620174812
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [ ]:
# Використовуємо алгоритм, який дає найкращий rmse:
algo = gs.best_estimator["rmse"]
algo.fit(data.build_full_trainset())

In [ ]:
# Знаходимо рекоендації для того самого користувача
user_id = '83'
top_n = 10

recommendations = []
for i in range(1, 1683):  # 1682 - кількість фільмів
    prediction = algo.predict(user_id, str(i))
    recommendations.append((i, prediction.est))

In [ ]:
recommendations.sort(key=lambda x: x[1], reverse=True)
for movie_id, rating in recommendations[:top_n]:
    print(f"Фільм ID: {movie_id}, Оцінка: {rating}")

Фільм ID: 408, Оцінка: 4.273667841964581
Фільм ID: 169, Оцінка: 4.242788201743192
Фільм ID: 318, Оцінка: 4.234443049858371
Фільм ID: 64, Оцінка: 4.2092407680208215
Фільм ID: 483, Оцінка: 4.177969733379412
Фільм ID: 114, Оцінка: 4.177480525255436
Фільм ID: 12, Оцінка: 4.1691847522571095
Фільм ID: 603, Оцінка: 4.142908724340256
Фільм ID: 178, Оцінка: 4.134403609752364
Фільм ID: 50, Оцінка: 4.132163552216122


**Підсумок**.

Реалізували 2 алгоритма SVD та KNN. SVD показав кращі результати на основі кросвалідації за MAE, тому орали його спершу для передбачення рекомендацій для користувача 83.

Далі реалізували алгоритм SVD з пошуком парметрів через GridSearch на основі RMSE і знову передбачили рекомендації для того самого користувача.

Лише 4 тих самих фільмів маємо, що і в результаті алгоритму з крос-валідацією.

## Завдання 2. Власна рекомендаційна система

In [ ]:
from surprise import AlgoBase
from surprise import PredictionImpossible

In [ ]:
# Визначаємо клас MyOwnAlgorithm
class MyOwnAlgorithm(AlgoBase):
    def __init__(self, sim_options={}, bsl_options={}, n_epochs=10, lr_all=0.005, reg_all=0.4):
        AlgoBase.__init__(self, sim_options=sim_options, bsl_options=bsl_options)
        self.n_epochs = n_epochs
        self.lr_all = lr_all
        self.reg_all = reg_all

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)

        # Обчислюємо базові лінії та схожості
        self.bu, self.bi = self.compute_baselines()
        self.sim = self.compute_similarities()

        return self

    def estimate(self, u, i):
        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible("User and/or item is unknown.")

        # Обчислюємо схожості між користувачем u і іншими користувачами, що також оцінили цей фільм
        neighbors = [(v, self.sim[u, v]) for (v, r) in self.trainset.ir[i]]
        # Сортуємо сусідів за схожістю
        neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

        print("The 3 nearest neighbors of user", str(u), "are:")
        for v, sim_uv in neighbors[:3]:
            print(f"user {v} with sim {sim_uv:1.2f}")

        # Повертаємо базову оцінку (baseline estimate)
        bsl = self.trainset.global_mean + self.bu[u] + self.bi[i]
        return bsl

In [ ]:
# Завантажуємо датасет
data = Dataset.load_builtin('ml-100k')

In [ ]:
# Розділяємо на навчальний і тестовий набір
trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
# Створюємо екземпляр власного алгоритму
algo = MyOwnAlgorithm()

In [ ]:
# Навчаємо модель
algo.fit(trainset)

# Прогнозуємо результати
predictions = algo.test(testset)

# Оцінюємо точність за MAE
accuracy.mae(predictions)

Streaming output truncated to the last 5000 lines.
user 892 with sim 1.00
user 405 with sim 1.00
The 3 nearest neighbors of user 276 are:
user 524 with sim 0.70
user 36 with sim 0.50
user 60 with sim 0.47
The 3 nearest neighbors of user 291 are:
user 103 with sim 0.73
user 657 with sim 0.60
user 398 with sim 0.57
The 3 nearest neighbors of user 197 are:
user 448 with sim 0.58
user 3 with sim 0.57
user 326 with sim 0.56
The 3 nearest neighbors of user 246 are:
user 849 with sim 1.00
user 934 with sim 0.60
user 754 with sim 0.50
The 3 nearest neighbors of user 275 are:
user 628 with sim 0.67
user 498 with sim 0.63
user 186 with sim 0.63
The 3 nearest neighbors of user 249 are:
user 837 with sim 0.43
user 454 with sim 0.43
user 28 with sim 0.42
The 3 nearest neighbors of user 690 are:
user 455 with sim 0.61
user 557 with sim 0.60
user 83 with sim 0.50
The 3 nearest neighbors of user 96 are:
user 895 with sim 0.64
user 6 with sim 0.52
user 110 with sim 0.52
The 3 nearest neighbors of user 

0.747062256843964

**Висновок**. Власний алгоритм отрима оцінку MAE - 0.747, що є більши за оцінку MAE 0.7402 з вбудованим алгоритмом SVD запущеним раніше. Отже, алгоритм SVD буде оптимальнішим для вирішення даної задачі.

Далі реалізуємо пошук по гіперпараметрам вручну, оскільки для surpise ми не можемо застосувати наш власний алгоритм, щоб порівняти результат алгоритму SVD з GredSearch та власний алгоритм з підбором параметрів.

In [ ]:
# Пошук гіперпараметрів вручну для власного алгоритму
param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
}

best_score = float('inf')
best_params = None

for n_epoch in param_grid['n_epochs']:
    for lr in param_grid['lr_all']:
        for reg in param_grid['reg_all']:
            # Створюємо і налаштовуємо алгоритм з поточними параметрами
            algo = MyOwnAlgorithm(n_epochs=n_epoch, lr_all=lr, reg_all=reg)
            algo.fit(trainset)

            # Прогнозуємо результати
            predictions = algo.test(testset)

            # Оцінюємо точність за RMSE
            score = accuracy.rmse(predictions, verbose=False)

            if score < best_score:
                best_score = score
                best_params = {"n_epochs": n_epoch, "lr_all": lr, "reg_all": reg}

print("Найкраща оцінка RMSE:", best_score)
print("Найкращі параметри:", best_params)

Streaming output truncated to the last 5000 lines.
user 405 with sim 1.00
The 3 nearest neighbors of user 276 are:
user 524 with sim 0.70
user 36 with sim 0.50
user 60 with sim 0.47
The 3 nearest neighbors of user 291 are:
user 103 with sim 0.73
user 657 with sim 0.60
user 398 with sim 0.57
The 3 nearest neighbors of user 197 are:
user 448 with sim 0.58
user 3 with sim 0.57
user 326 with sim 0.56
The 3 nearest neighbors of user 246 are:
user 849 with sim 1.00
user 934 with sim 0.60
user 754 with sim 0.50
The 3 nearest neighbors of user 275 are:
user 628 with sim 0.67
user 498 with sim 0.63
user 186 with sim 0.63
The 3 nearest neighbors of user 249 are:
user 837 with sim 0.43
user 454 with sim 0.43
user 28 with sim 0.42
The 3 nearest neighbors of user 690 are:
user 455 with sim 0.61
user 557 with sim 0.60
user 83 with sim 0.50
The 3 nearest neighbors of user 96 are:
user 895 with sim 0.64
user 6 with sim 0.52
user 110 with sim 0.52
The 3 nearest neighbors of user 205 are:
user 681 with 

Найкращі параметри такі ж як і для SVD, окрін lr_all. Сам RMSE нижчий у власного алгоритма - 0.94, тоді як в SVD - 0.96

У підсумку найоптимальніший алгоритм - SVD з підбором параметрів за допомогою GridSearchCV.